In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import coolpuppy
import glob
import os
import seaborn as sns
import bioframe as bf

In [2]:
ENCFF048WDN = pd.read_csv("ENCFF048DWN.bed", sep="\t", header=None)
ENCFF048WDN.columns = ["chrom", "start", "end", "name",  "score", "strand", "signalValue", "pValue", "qValue", "peak"]

In [3]:
ENCFF048WDN.loc[ENCFF048WDN["signalValue"] <= np.quantile(ENCFF048WDN["signalValue"], 0.25), "quartile"] = 1
ENCFF048WDN.loc[ENCFF048WDN["signalValue"] > np.quantile(ENCFF048WDN["signalValue"], 0.25), "quartile"] = 2
ENCFF048WDN.loc[ENCFF048WDN["signalValue"] > np.quantile(ENCFF048WDN["signalValue"], 0.5), "quartile"] = 3
ENCFF048WDN.loc[ENCFF048WDN["signalValue"] > np.quantile(ENCFF048WDN["signalValue"], 0.75), "quartile"] = 4

In [4]:
for i in range(1,5):
    ENCFF048WDN.loc[ENCFF048WDN["quartile"] == i, 
                    ["chrom", "start", "end"]].to_csv("ENCFF048WDN_DNase_quartile_"+str(i)+"_mm10.bed", 
                                                      sep="\t", 
                                                      header=None, 
                                                      index=False)

In [5]:
!for f in ENCFF048WDN_DNase_quartile_?_mm10.bed; do \
bedtools closest -a $f -b ../mm10.refGene_exon1_TSS.bed -d | \
awk -v OFS="\t" '{if($10<=1000) {print $1,$2,$3}}' | \
wc -l ; done

2778
2705
3511
8522


In [6]:
!for f in ENCFF048WDN_DNase_quartile_?_mm10.bed; do \
bedtools closest -a $f -b ../mm10.refGene_exon1_TSS.bed -d | \
awk -v OFS="\t" '{if($10<=1000) {print $1,$2,$3}}' | \
shuf -n 2705 > "${f/.bed/_1kbwithinTSS_shuf2705.bed}" ; done

In [7]:
!for f in ENCFF048WDN_DNase_quartile_?_mm10.bed; do \
bedtools closest -a $f -b ../mm10.refGene_exon1_TSS.bed -d | \
awk -v OFS="\t" '{if($10>5000) {print $1,$2,$3}}' | \
shuf -n 2705 > "${f/.bed/_5kbfromTSS_shuf2705.bed}" ; done